<a href="https://colab.research.google.com/github/armandossrecife/mysummary/blob/main/quick_process_text_summary_using_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Hibrida para resumo de textos

Usando a API do Google Gemini e Transformers do Hugging Face

Como ler o conteúdo de um arquivo pdf e fazer o resumo desse conteudo em Português.

1. Ler o conteúdo em português
2. Traduz o conteúdo de português para inglês (o Transformers só reconhece texto em inglês)
3. Faz o resumo do texto em inglês
4. Traduz o resumo de inglês para português
5. Exibe o resultado do resumo em português

## Instale o SDK da API do Gemini e o Transformers do Hugging Face

The Python SDK for the Gemini API is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package.

Also, install [PyPDF2](https://pypdf2.readthedocs.io/en/3.x/)

Install the dependency using pip:

In [1]:
!pip install -q -U google-generativeai

In [2]:
!pip install transformers

In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.4 MB/s eta 0:00:00


## Configure a sua chave da API do Gemini

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

Obs: nunca publique sua chave no repositório

In [4]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Inicialize o Modelo Generativo

Before you can make any API calls, you need to initialize the Generative Model.

In [5]:
model = genai.GenerativeModel('gemini-pro')

## Dado um documento, faz a tradução de português para inglês

O texto pode ser extraído de um arquivo .pdf

In [6]:
!wget https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/d1.pdf

--2024-10-19 15:27:47--  https://raw.githubusercontent.com/armandossrecife/mysummary/refs/heads/main/pdfs/d1.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1850029 (1.8M) [application/octet-stream]
Saving to: ‘d1.pdf’

d1.pdf              100%[===================>]   1.76M  --.-KB/s    in 0.07s   

2024-10-19 15:27:47 (25.5 MB/s) - ‘d1.pdf’ saved [1850029/1850029]



In [7]:
import PyPDF2
import os
from datetime import datetime
# Load pipeline from transformers
from transformers import pipeline

def extract_pdf_text(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            pdf_text = {}
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                pdf_text[page_num + 1] = text
            return pdf_text
    except FileNotFoundError:
        print(f"Error: PDF file not found at {pdf_path}")
    except PermissionError:
        print(f"Error: Permission denied to access PDF file at {pdf_path}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

def request_to_gemini(prompt_text):
  print("Aguarde...")
  t1 = datetime.now()
  response_texto = model.generate_content(prompt_text)
  t2 = datetime.now()
  print(response_texto)
  tempo_resposta = t2 - t1
  print(f"Tempo de resposta: {tempo_resposta}")
  return response_texto

def get_text_from_response(response_texto):
  try:
    if len(response_texto.candidates) > 0:
      summary_content = response_texto.candidates[0].content
      if len(summary_content.parts) > 0:
        summary_text = summary_content.parts[0].text
        return summary_text  # Return the extracted text
      else:
        print("No content parts found in the first candidate.")
    else:
      print("Nenhuma resposta disponível")
  except (AttributeError, IndexError):
    print("Error accessing response data. Please check the response structure.")

In [8]:
texto_documento = ""

meu_conteudo =  extract_pdf_text(pdf_path="d1.pdf")

In [9]:
print(f"Quantidade de páginas: {len(meu_conteudo.keys())}")

Quantidade de páginas: 25


In [10]:
for k,v in meu_conteudo.items():
  print(f"Página: {k}")
  print(f"Texto: {v}")
  texto_documento = texto_documento + v

Página: 1
Texto: AO CHEFE DO DEPARTAMENTO DE COMPUTAÇÃO  DA UNIVERSIDADE FEDERAL DO PIAUÍ  Caro Chefe de Departamento, Eu Armando Soares Sousa, Matrícula UFPI 1886865 lotado no Departamento de Computação do Centro de Ciência da Natureza da Universidade Federal do Piauí solicita sua concordância quanto minha PRORROGAÇÃO DE AFASTAMENTO a fim de dar continuidade ao curso de Pós-Graduação em Ciência da Computação em nível de Doutorado na (Área de Concentração: Ciência da Computação) na Universidade Federal do Ceará, por mais 6 (seis) meses, a partir de 23/02/2022, em conformidade com a Resolução No. 186/06-CEPEX. Além disso, gostaria que o senhor anexasse a meu pedido o documento referente ao plano de qualificação 2022 aprovado na Assembleia Departamental informando a deliberação favorável a meu pedido de afastamento em 2022 para poder concluir minhas atividades do doutorado. Fortaleza, 21, de Dezembro de 2021  __________________________________  (Assinatura) 
Solicitação ao Chefe para P

In [11]:
texto_documento

'AO CHEFE DO DEPARTAMENTO DE COMPUTAÇÃO  DA UNIVERSIDADE FEDERAL DO PIAUÍ  Caro Chefe de Departamento, Eu Armando Soares Sousa, Matrícula UFPI 1886865 lotado no Departamento de Computação do Centro de Ciência da Natureza da Universidade Federal do Piauí solicita sua concordância quanto minha PRORROGAÇÃO DE AFASTAMENTO a fim de dar continuidade ao curso de Pós-Graduação em Ciência da Computação em nível de Doutorado na (Área de Concentração: Ciência da Computação) na Universidade Federal do Ceará, por mais 6 (seis) meses, a partir de 23/02/2022, em conformidade com a Resolução No. 186/06-CEPEX. Além disso, gostaria que o senhor anexasse a meu pedido o documento referente ao plano de qualificação 2022 aprovado na Assembleia Departamental informando a deliberação favorável a meu pedido de afastamento em 2022 para poder concluir minhas atividades do doutorado. Fortaleza, 21, de Dezembro de 2021  __________________________________  (Assinatura) \nSolicitação ao Chefe para Prorrogac ̧ão d

### Envia o conteúdo para o Google Gemini traduzir

In [12]:
pergunte_ao_gemini = "Translate the following text from portuguese to english: "
prompt_text = f"{pergunte_ao_gemini}{texto_documento}"
response_text_in_english = request_to_gemini(prompt_text)

Aguarde...
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "TO THE HEAD OF THE COMPUTER SCIENCE DEPARTMENT   OF THE FEDERAL UNIVERSITY OF PIAU\u00cd   Dear Head of the Department, I, Armando Soares Sousa, Registration UFPI 1886865, assigned to the Computer Science Department of the Center for Natural Science of the Federal University of Piau\u00ed, request your approval for the EXTENSION OF MY LEAVE OF ABSENCE in order to continue the Postgraduate course in Computer Science at the Doctoral level in the (Concentration Area: Computer Science) at the Federal University of Cear\u00e1, for an additional 6 (six) months, starting on 02/23/2022, in accordance with Resolution No. 186/06-CEPEX. Furthermore, I would like you to attach to my request the document referring to the 2022 qualifying plan approved by the De

In [13]:
minha_traducao_01 = get_text_from_response(response_text_in_english)
print(minha_traducao_01)

TO THE HEAD OF THE COMPUTER SCIENCE DEPARTMENT   OF THE FEDERAL UNIVERSITY OF PIAUÍ   Dear Head of the Department, I, Armando Soares Sousa, Registration UFPI 1886865, assigned to the Computer Science Department of the Center for Natural Science of the Federal University of Piauí, request your approval for the EXTENSION OF MY LEAVE OF ABSENCE in order to continue the Postgraduate course in Computer Science at the Doctoral level in the (Concentration Area: Computer Science) at the Federal University of Ceará, for an additional 6 (six) months, starting on 02/23/2022, in accordance with Resolution No. 186/06-CEPEX. Furthermore, I would like you to attach to my request the document referring to the 2022 qualifying plan approved by the Departmental Assembly, informing the favorable decision to approve my request for leave in 2022 so that I can complete my doctoral activities. Fortaleza, December 21, 2021   __________________________________   (Signature)   REQUEST FOR EXTENSION OF ABSENCE 1D

## Faz o resumo do texto em inglês

Usando o Transformers do Hugging Face

### Carrega o pipeline do Hugging Face

In [14]:
# Load the summarization pipeline. Set facebook/bart-large-cnn as the model to use for summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
len(minha_traducao_01)

7817

In [18]:
teste_parte_traducao = minha_traducao_01[:3000]

In [19]:
my_summary = summarizer(teste_parte_traducao)

In [20]:
# Print the summary
print(my_summary[0]['summary_text'])

I, Armando Soares Sousa, want to continue the Postgraduate course in Computer Science at the Doctoral level in the (Concentration Area: Computer Science) at the Federal University of Ceará, for an additional 6 (six) months, starting on 02/23/2022. I would like to inform you that in addition to the "Request for Approval from the Head of the Department of Computer Science", the following documents are attached.


## Traduz o resumo para o português

In [21]:
summary_in_english = my_summary[0]['summary_text']
print(type(summary_in_english))
print(summary_in_english)

<class 'str'>
I, Armando Soares Sousa, want to continue the Postgraduate course in Computer Science at the Doctoral level in the (Concentration Area: Computer Science) at the Federal University of Ceará, for an additional 6 (six) months, starting on 02/23/2022. I would like to inform you that in addition to the "Request for Approval from the Head of the Department of Computer Science", the following documents are attached.


In [22]:
pergunte_ao_gemini2 = "Translate the following text from english to portuguese: "
prompt_text2 = f"{pergunte_ao_gemini2}{summary_in_english}"
response_summary_in_english = request_to_gemini(prompt_text2)

Aguarde...
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Eu, Armando Soares Sousa, solicito prorroga\u00e7\u00e3o de prazo do curso de P\u00f3s-Gradua\u00e7\u00e3o em Ci\u00eancia da Computa\u00e7\u00e3o, n\u00edvel Doutorado, na \u00c1rea de Concentra\u00e7\u00e3o: Ci\u00eancia da Computa\u00e7\u00e3o, na Universidade Federal do Cear\u00e1, por mais 6 (seis) meses, a partir de 23/02/2022. Esclare\u00e7o que al\u00e9m do \"Requerimento para Prorroga\u00e7\u00e3o Submetido ao Chefe do Departamento de Ci\u00eancia da Computa\u00e7\u00e3o\" segue anexo os documentos."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        

In [23]:
get_text_from_response(response_texto=response_summary_in_english)

'Eu, Armando Soares Sousa, solicito prorrogação de prazo do curso de Pós-Graduação em Ciência da Computação, nível Doutorado, na Área de Concentração: Ciência da Computação, na Universidade Federal do Ceará, por mais 6 (seis) meses, a partir de 23/02/2022. Esclareço que além do "Requerimento para Prorrogação Submetido ao Chefe do Departamento de Ciência da Computação" segue anexo os documentos.'